In [ ]:
#IMPORTA AS BIBLIOTECAS NECESSÁRIAS
import pandas as pd
import requests
from bs4 import BeautifulSoup
import warnings
import re
import csv

In [ ]:
#ABRE UMA SESSION PARA CONEXÃO COM O SITE DE CONSULTA DE ACÓRDÃOS DO CARF
sessionCarf = requests.Session()
urlCarf = "https://carf.fazenda.gov.br/sincon/public/pages/ConsultarJurisprudencia/consultarJurisprudenciaCarf.jsf"
warnings.filterwarnings('ignore')
htmlCarf = sessionCarf.get(urlCarf, verify=False)

In [ ]:
#CRIA UM OBJETO BEAUTIFULSOUP DA PÁGINA
soupCarf = BeautifulSoup(htmlCarf.text, 'html.parser')

In [ ]:
#BUSCA O VIEWSTATE ID DA PÁGINA
vwIdCarf = soupCarf.find_all(id='javax.faces.ViewState')[0]['value']

In [ ]:
#CRIA DICIONÁRIO DE PARÂMETROS PARA CONSULTA DE ACÓRDÃOS DE COFINS ENTRE 2016 E 2020
paramCarf = {
    "consultaJurisprudenciaForm": "consultaJurisprudenciaForm"
    , "dataInicialInputDate": "01/2016"
    , "dataFinalInputDate": "12/2020"
    , "uniqueToken": ""
    , "campo_pesquisa1": "1"
    , "campo_pesquisa2": "1"
    , "campo_pesquisa3": "1"
    , "valor_pesquisa3": "cofins"
    , "AJAXREQUEST": "_viewRoot"
    , "javax.faces.ViewState": vwIdCarf
    , "j_id51": "j_id51"
}

In [ ]:
#EXECUTA A CONSULTA DE ACÓRDÃOS COM BASE NOS PARÂMETROS
jsidCarf = sessionCarf.cookies.get_dict()['JSESSIONID']
urlCarfJSId = "https://carf.fazenda.gov.br/sincon/public/pages/ConsultarJurisprudencia/consultarJurisprudenciaCarf.jsf;jsessionid=" + jsidCarf
htmlCarf = requests.post(urlCarfJSId, paramCarf, verify=False)

In [ ]:
#CRIA OBJETO SOUP DA PÁGINA DE RESULTADOS DA CONSULTA
soupResultCarf = BeautifulSoup(htmlCarf.text, 'html.parser')
vwIdResultCarf = soupResultCarf.find_all(id='javax.faces.ViewState')[0]['value']

In [ ]:
#VERIFICA O NÚMERO DO PÁGINAS DE RESULTADO PARA QUE SEJA ACESSADA UMA POR UMA POSTERIORMENTE
tabPags = soupResultCarf.find(id="dataScroller_1_table")
mNumPags = re.search("Página 1 de ([0-9]+)", tabPags.text)
numPags = int(mNumPags.group(1))

In [ ]:
#CRIA UMA LISTA QUE IRÁ ARMAZENAR OS DADOS COLETADOS DE ACADA ACÓRDÃO
listaDadosColetados = []

In [ ]:
#VISITA PÁGINA POR PÁGINA DO RESULTADO DA CONSULTA, ENTRANDO NA PÁGINA DE DETALHAMENTO DE CADA ACÓRDÃO
#E EXTRAINDO OS DADOS NECESSÁRIOS, ARMAZENANDO-OS NA LISTA DE DADOS COLETADOS
soupPagResult = soupResultCarf
vwIdPagResult = vwIdResultCarf

numProxAcordao = 0
for p in range(numPags):
    print("Pág " + str(p + 1) + " de " + str(numPags))
    
    numAcordaosPag = len(soupPagResult.find_all(id = re.compile("tblJurisprudencia:[0-9]+:numDecisao")))
    
    for a in range(numAcordaosPag):
        vwIdPagResult = soupPagResult.find_all(id='javax.faces.ViewState')[0]['value']
        paramAcordao = {
            "formAcordaos": "formAcordaos"
            , "uniqueToken":""
            , "javax.faces.ViewState": vwIdPagResult
            , "tblJurisprudencia:{}:numDecisao".format(numProxAcordao): "tblJurisprudencia:{}:numDecisao".format(numProxAcordao)
        }
        
        htmlAcordao = requests.post("https://carf.fazenda.gov.br/sincon/public/pages/ConsultarJurisprudencia/listaJurisprudenciaCarf.jsf;jsessionid=" + jsidCarf, paramAcordao, verify=False)
        
        soupAcordao = BeautifulSoup(htmlAcordao.text, 'html.parser')
        vwIdAcordao = soupAcordao.find_all(id='javax.faces.ViewState')[0]['value']
        
        nomeContribuinte = ""
        objNomeContribuinte = soupAcordao.find(id="formAcordaos:contribuinte")
        if objNomeContribuinte != None:
            nomeContribuinte = objNomeContribuinte.text
        
        tipoRecurso = ""
        objTipoRecurso = soupAcordao.find(id="formAcordaos:labeTipoRecurso")
        if objTipoRecurso != None:
            tipoRecurso = objTipoRecurso.parent.find_all("span")[1].text
        
        dataSessao = ""
        objDataSessao = soupAcordao.find(id="formAcordaos:dataSessao")
        if objDataSessao != None:
            dataSessao = objDataSessao.text
        
        ementa = ""
        objEmenta = soupAcordao.find(id="formAcordaos:ementa")
        if objEmenta != None:
            ementas = objEmenta.contents
            ementa = "".join([str(item) for item in ementas])
        
        listaDadosAcordaos = [nomeContribuinte, tipoRecurso, dataSessao, ementa]
        listaDadosColetados.append(listaDadosAcordaos)
        
        nameVoltar = soupAcordao.find(value="Voltar")["name"]
        paramRetPagResult = {
            "formAcordaos": "formAcordaos"
            , "uniqueToken":""
            , nameVoltar: "Voltar"
            , "javax.faces.ViewState": vwIdAcordao
            , "formAcordaos:_idcl": ""
        }
        
        htmlRetPagResult = requests.post("https://carf.fazenda.gov.br/sincon/public/pages/ConsultarJurisprudencia/listaJurisprudencia.jsf;jsessionid=" + jsidCarf, paramRetPagResult, verify=False)

        soupPagResult = BeautifulSoup(htmlRetPagResult.text, 'html.parser')        
        vwIdPagResult = soupPagResult.find_all(id='javax.faces.ViewState')[0]['value']
        
        numProxAcordao = numProxAcordao + 1

    paramProxPagResult = {
        "AJAXREQUEST": "_viewRoot",
        "formAcordaos": "formAcordaos",
        "uniqueToken": "",
        "javax.faces.ViewState": vwIdPagResult,
        "ajaxSingle": "dataScroller_2",
        "dataScroller_2": "next",
        "AJAX:EVENTS_COUNT": "1"
    }
    
    htmlProxPagResult = requests.post("https://carf.fazenda.gov.br/sincon/public/pages/ConsultarJurisprudencia/listaJurisprudenciaCarf.jsf;jsessionid=" + jsidCarf, paramProxPagResult, verify=False)
    soupPagResult = BeautifulSoup(htmlProxPagResult.text, 'html.parser')        
    vwIdPagResult = soupPagResult.find_all(id='javax.faces.ViewState')[0]['value']

In [ ]:
#CRIA UM DATAFRAME PANDAS A PARTIR DA LISTA DE DADOS COLETADOS, POSTERIORMENTE EXPORTA O DATAFRAME PARA UM ARQUIVO CSV
#CRIANDO ENTÃO, O DATASET DE ACÓRDÃOS DE COFINS DO CARF
dfDadosColetados = pd.DataFrame(listaDadosColetados, columns=["NOME CONTRIBUINTE", "TIPO RECURSO"
    , "DATA SESSAO", "EMENTA"])
dfDadosColetados.to_csv('../Dataset CARF COFINS/DADOS ACORDAOS COFINS CARF 2016-2020.csv', index=False, sep="#")